# Forehead Wrinkle Matching
We perform image matching using some popular methods like SIFT, SURF, ArcFace, ORB, AKAZE, BRISK and SSIM. 

In [ ]:
!pip uninstall opencv-python
!pip install opencv-contrib-python==3.4.2.17

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
from natsort import natsorted
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def img_enhance(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img, (3, 3), 0)
    clahe = cv2.createCLAHE(clipLimit=5.0)
    img = clahe.apply(img)
    return img

In [ ]:
def calc_scores(thresh, genuine, imposter):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for g in genuine:
        if g < thresh:
            fn += 1
        else:
            tp += 1
    
    for i in imposter:
        if i < thresh:
            tn += 1
        else:
            fp += 1
    return (tp+tn)/(tp+tn+fp+fn), fp/(fp+tn), fn/(fn+tp)

In [ ]:
threshes = np.arange(0, 1, 0.0001).tolist()

# SIFT

In [ ]:
file1 = open("/content/drive/MyDrive/Colab Notebooks/output_SIFT.csv", "w")
path1 = '/content/drive/MyDrive/Colab Notebooks/forehead_dataset/'

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()
bf = cv2.BFMatcher()
for images in natsorted(os.listdir(path1)):
    str1 = str(images)
    img1 = os.path.join(path1, images)
    img1 = cv2.imread(img1)
    img1 = img_enhance(img1)
    kp1, des1 = sift.detectAndCompute(img1,None)
    if len(kp1)>0:
       for image in natsorted(os.listdir(path1)):
           img2 = os.path.join(path1, image)
           str2 = str(image)
           if (str1.split("_")[2]==str2.split("_")[2]):
               continue
           if (img1 != img2):
               img2 = cv2.imread(img2)
               img2 = img_enhance(img2)
               kp2, des2 = sift.detectAndCompute(img2,None)
          
               if des1 is None:
                 file1.write( str1 + " " + str2 + " " + "0" +"\n")
                 print(str1 + " " + str2 + " " + str(score))
                 continue
               if des2 is None:
                 file1.write( str1 + " " + str2 + " " + "0"+"\n")
                 print(str1 + " " + str2 + " " + str(score))
                 continue
               matches = bf.knnMatch(des1,des2, k=2)
               dis = bf.match(des1,des2)
            #----------------------------------------------
               number_keypoints = 0
               if len(kp1) <= len(kp2):
                   number_keypoints = len(kp1)
               else:
                   number_keypoints = len(kp2)
               good = []
               try:
                 for m,n in matches:
                     if m.distance < 0.9*n.distance:
                         good.append([m])
               except:
                 pass 
               score = len(good) / number_keypoints
               if(score>1):
                 file1.write( str1 + " " + str2 + " " + "1.0"+"\n")
                 print(str1 + " " + str2 + " " + "1.0")
               else:
                 file1.write( str1 + " " + str2 + " " + str(score)+"\n")
                 print(str1 + " " + str2 + " " + str(score))

In [ ]:
file1.close()

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/output_SIFT.csv", header=None)
df

In [ ]:
genuine = []
imposter = []

In [ ]:
for i in df[0]:
  print(i)
  str1 = i.split()
  a = str1[0].split("_") 
  b = str1[1].split("_")
  if (a[0]==b[0]):
    genuine.append(float(str1[2]))
  else:
    imposter.append(float(str1[2]))

In [ ]:
bins = np.linspace(0, 1, 100)
from matplotlib import pyplot
pyplot.hist(imposter, bins, alpha=0.25, label='imposter', weights=np.ones(len(imposter)) / len(imposter), histtype = 'stepfilled', align = 'mid')
pyplot.hist(genuine, bins, alpha=0.5, label='genuine', weights=np.ones(len(genuine)) / len(genuine), histtype = 'stepfilled')
pyplot.legend(loc='upper right')
pyplot.show()

In [ ]:
scores_sift = {}
crr_sift = []
far_sift = []
frr_sift = []
eer_thresh_sift = 1000
eer_diff_sift = 1000
crrSift = 10000
for thresh in threshes:
    scores_sift[thresh] = calc_scores(thresh, genuine, imposter)
    crr_sift.append(scores_sift[thresh][0])
    far_sift.append(scores_sift[thresh][1])
    frr_sift.append(scores_sift[thresh][2])
    if(abs(scores_sift[thresh][1]-scores_sift[thresh][2])<eer_diff_sift):
      eer_diff_sift = abs(scores_sift[thresh][1]-scores_sift[thresh][2])
      eer_thresh_sift = thresh
      crrSift = scores_sift[thresh][0]
plt.plot(far_sift, frr_sift)
plt.xlabel('False Acceptance Rate')
plt.ylabel('False Rejection Rate')
plt.show()
print("eer_diff=",eer_diff_sift," eer_thresh=",eer_thresh_sift)

In [ ]:
crrSift

# SURF

In [ ]:
file1 = open("/content/drive/MyDrive/Colab Notebooks/output_SURF.csv", "w")
path1 = '/content/drive/MyDrive/Colab Notebooks/forehead_dataset/'

In [ ]:
surf = cv2.xfeatures2d.SURF_create(400)
bf = cv2.BFMatcher()
for images in natsorted(os.listdir(path1)):
    str1 = str(images)
    img1 = os.path.join(path1, images)
    #print(img1)
    img1 = cv2.imread(img1)
    img1 = img_enhance(img1)
    kp1, des1 = surf.detectAndCompute(img1,None)
    if len(kp1)>0:
       for image in natsorted(os.listdir(path1)):
           img2 = os.path.join(path1, image)
           str2 = str(image)
           if (str1.split("_")[2]==str2.split("_")[2]):
               #print(str1.split("_"))
               continue
           if (img1 != img2):
               img2 = cv2.imread(img2)
               img2 = img_enhance(img2)
               kp2, des2 = surf.detectAndCompute(img2,None)
          
               if des1 is None:
                 file1.write( str1 + " " + str2 + " " + "0" +"\n")
                 print(str1 + " " + str2 + " " + str(score))
                 continue
               if des2 is None:
                 file1.write( str1 + " " + str2 + " " + "0"+"\n")
                 print(str1 + " " + str2 + " " + str(score))
                 continue
              #  print(des1)
              #  print(des2)
              #  print(type(des1),type(des2))
               matches = bf.knnMatch(des1,des2, k=2)
               dis = bf.match(des1,des2)
            #----------------------------------------------
               number_keypoints = 0
               if len(kp1) <= len(kp2):
                   number_keypoints = len(kp1)
               else:
                   number_keypoints = len(kp2)
               good = []
               #print(matches)
               try:
                 for m,n in matches:
                     if m.distance < 0.9*n.distance:
                         good.append([m])
               except:
                 pass 
               score = len(good) / number_keypoints
               if(score>1):
                 file1.write( str1 + " " + str2 + " " + "1.0"+"\n")
                 print(str1 + " " + str2 + " " + "1.0")
               else:
                 file1.write( str1 + " " + str2 + " " + str(score)+"\n")
                 print(str1 + " " + str2 + " " + str(score))

In [ ]:
file1.close()
genuine = []
imposter = []
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/output_SURF.csv", header=None)
for i in df[0]:
  print(i)
  str1 = i.split()
  a = str1[0].split("_") 
  b = str1[1].split("_")
  if (a[0]==b[0]):
    genuine.append(float(str1[2]))
  else:
    imposter.append(float(str1[2]))

In [ ]:
bins = np.linspace(0, 1, 50)
from matplotlib import pyplot
pyplot.hist(imposter, bins, alpha=0.25, label='imposter', weights=np.ones(len(imposter)) / len(imposter), histtype = 'stepfilled', align = 'mid')
pyplot.hist(genuine, bins, alpha=0.5, label='genuine', weights=np.ones(len(genuine)) / len(genuine), histtype = 'stepfilled')
pyplot.legend(loc='upper right')
pyplot.show()

In [ ]:
scores_surf = {}
crr_surf = []
far_surf = []
frr_surf = []
eer_thresh_surf = 1000
eer_diff_surf = 1000
crrSurf = 1000
for thresh in threshes:
    scores_surf[thresh] = calc_scores(thresh, genuine, imposter)
    crr_surf.append(scores_surf[thresh][0])
    far_surf.append(scores_surf[thresh][1])
    frr_surf.append(scores_surf[thresh][2])
    if(abs(scores_surf[thresh][1]-scores_surf[thresh][2])<eer_diff_surf):
      eer_diff_surf = abs(scores_surf[thresh][1]-scores_surf[thresh][2])
      eer_thresh_surf = thresh
      crrSurf = scores_surf[thresh][0]
plt.plot(far_surf, frr_surf)
plt.xlabel('False Acceptance Rate')
plt.ylabel('False Rejection Rate')
plt.show()
print("eer_diff=",eer_diff_surf," eer_thresh=",eer_thresh_surf)

In [ ]:
crrSurf

# ArcFace

In [ ]:
!pip install arcface

In [ ]:
from arcface import ArcFace
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance

In [ ]:
file1 = open("/content/drive/MyDrive/Colab Notebooks/output_arcface.csv", "w")
path1 = '/content/drive/MyDrive/Colab Notebooks/forehead_dataset/'

In [ ]:
arcface = ArcFace.ArcFace()
for images in natsorted(os.listdir(path1)):
    str1 = str(images)
    if str1=='.ipynb_checkpoints':
        continue
    img1 = os.path.join(path1, images)
    img1 = cv2.imread(img1)
    emb1 = arcface.calc_emb(img1)
    if len(emb1)>0:
       for image in natsorted(os.listdir(path1)):
           img2 = os.path.join(path1, image)
           str2 = str(image)
           if str1 == '.ipynb_checkpoints' or str2 == '.ipynb_checkpoints':
               continue
           if (str1.split("_")[2]==str2.split("_")[2]):
               continue
           if (img1 != img2):
               img2 = cv2.imread(img2)
               emb2 = arcface.calc_emb(img2)
          
               if emb1 is None:
                 file1.write(str1 + " " + str2 + " " + str(0)+"\n")
                 print(str1 + " " + str2 + " " + str(0))
                 continue
               if emb2 is None:
                 file1.write(str1 + " " + str2 + " " + str(0)+"\n")
                 print(str1 + " " + str2 + " " + str(0))
                 continue
               dist = distance.cosine(emb1, emb2)
               file1.write(str1 + " " + str2 + " " + str(dist)+"\n")
               print(str1 + " " + str2 + " " + str(dist))

In [ ]:
file1.close()

In [ ]:
genuine = []
imposter = []
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/output_arcface.csv", header=None)
for i in df[0]:
  print(i)
  str1 = i.split()
  a = str1[0].split("_") 
  b = str1[1].split("_")
  if (a[0]==b[0]):
    genuine.append(1-float(str1[2]))
  else:
    imposter.append(1-float(str1[2]))

In [ ]:
bins = np.linspace(0, 1, 100)
from matplotlib import pyplot
pyplot.hist(imposter, bins, alpha=0.25, label='imposter', weights=np.ones(len(imposter)) / len(imposter), histtype = 'stepfilled', align = 'mid')
pyplot.hist(genuine, bins, alpha=0.5, label='genuine', weights=np.ones(len(genuine)) / len(genuine), histtype = 'stepfilled')
pyplot.legend(loc='upper right')
pyplot.show()

In [ ]:
scores_arc = {}
crr_arc = []
far_arc = []
frr_arc = []
eer_thresh_arc = 1000
eer_diff_arc = 1000
crrArc = 0
for thresh in threshes:
    scores_arc[thresh] = calc_scores(thresh, genuine, imposter)
    crr_arc.append(scores_arc[thresh][0])
    far_arc.append(scores_arc[thresh][1])
    frr_arc.append(scores_arc[thresh][2])
    if(abs(scores_arc[thresh][1]-scores_arc[thresh][2])<eer_diff_arc):
      eer_diff_arc = abs(scores_arc[thresh][1]-scores_arc[thresh][2])
      eer_thresh_arc = thresh
      crrArc = scores_arc[thresh][0]
plt.plot(far_arc, frr_arc)
plt.xlabel('False Acceptance Rate')
plt.ylabel('False Rejection Rate')
plt.show()
print("eer_diff=",eer_diff_arc," eer_thresh=",eer_thresh_arc)

In [ ]:
crrArc

# ORB

In [ ]:
file1 = open("/content/drive/MyDrive/Colab Notebooks/output_orb.csv", "w")
path1 = '/content/drive/MyDrive/Colab Notebooks/forehead_dataset/'

In [ ]:
orb = cv2.ORB_create()
bf = cv2.BFMatcher()
for images in natsorted(os.listdir(path1)):
    str1 = str(images)
    img1 = os.path.join(path1, images)
    #print(img1)
    img1 = cv2.imread(img1)
    img1 = img_enhance(img1)
    kp1, des1 = orb.detectAndCompute(img1,None)
    if len(kp1)>0:
       for image in natsorted(os.listdir(path1)):
           img2 = os.path.join(path1, image)
           str2 = str(image)
           if (str1.split("_")[2]==str2.split("_")[2]):
               #print(str1.split("_"))
               continue
           if (img1 != img2):
               img2 = cv2.imread(img2)
               img2 = img_enhance(img2)
               kp2, des2 = orb.detectAndCompute(img2,None)
          
               if des1 is None:
                 file1.write( str1 + " " + str2 + " " + "0" +"\n")
                 print(str1 + " " + str2 + " " + str(score))
                 continue
               if des2 is None:
                 file1.write( str1 + " " + str2 + " " + "0"+"\n")
                 print(str1 + " " + str2 + " " + str(score))
                 continue
              #  print(des1)
              #  print(des2)
              #  print(type(des1),type(des2))
               matches = bf.knnMatch(des1,des2, k=2)
               dis = bf.match(des1,des2)
            #----------------------------------------------
               number_keypoints = 0
               if len(kp1) <= len(kp2):
                   number_keypoints = len(kp1)
               else:
                   number_keypoints = len(kp2)
               good = []
               #print(matches)
               try:
                 for m,n in matches:
                     if m.distance < 0.9*n.distance:
                         good.append([m])
               except:
                 pass 
               score = len(good) / number_keypoints
               if(score>1):
                 file1.write( str1 + " " + str2 + " " + "1.0"+"\n")
                 print(str1 + " " + str2 + " " + "1.0")
               else:
                 file1.write( str1 + " " + str2 + " " + str(score)+"\n")
                 print(str1 + " " + str2 + " " + str(score))

In [ ]:
file1.close()
genuine = []
imposter = []
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/output_orb.csv", header=None)
for i in df[0]:
  print(i)
  str1 = i.split()
  a = str1[0].split("_") 
  b = str1[1].split("_")
  if (a[0]==b[0]):
    genuine.append(float(str1[2]))
  else:
    imposter.append(float(str1[2]))

In [ ]:
bins = np.linspace(0, 1, 100)
from matplotlib import pyplot
pyplot.hist(imposter, bins, alpha=0.25, label='imposter', weights=np.ones(len(imposter)) / len(imposter), histtype = 'stepfilled', align = 'mid')
pyplot.hist(genuine, bins, alpha=0.5, label='genuine', weights=np.ones(len(genuine)) / len(genuine), histtype = 'stepfilled')
pyplot.legend(loc='upper right')
pyplot.show()

In [ ]:
scores_orb = {}
crr_orb = []
far_orb = []
frr_orb = []
eer_thresh_orb = 1000000000000
eer_diff_orb = 1000000000000
crrOrb = 1000
for thresh in threshes:
    scores_orb[thresh] = calc_scores(thresh, genuine, imposter)
    crr_orb.append(scores_orb[thresh][0])
    far_orb.append(scores_orb[thresh][1])
    frr_orb.append(scores_orb[thresh][2])
    if(abs(scores_orb[thresh][1]-scores_orb[thresh][2])<eer_diff_orb):
      eer_diff_orb = abs(scores_orb[thresh][1]-scores_orb[thresh][2])
      eer_thresh_orb = thresh
      crrOrb = scores_orb[thresh][0]
plt.plot(far_orb, frr_orb)
plt.xlabel('False Acceptance Rate')
plt.ylabel('False Rejection Rate')
plt.show()
print("eer_diff=",eer_diff_orb," eer_thresh=",eer_thresh_orb)

In [ ]:
crrOrb

# AKAZE

In [ ]:
file1 = open("/content/drive/MyDrive/Colab Notebooks/output_akaze.csv", "w")
path1 = '/content/drive/MyDrive/Colab Notebooks/forehead_dataset/'

In [ ]:
akaze = cv2.AKAZE_create()
bf = cv2.BFMatcher()
for images in natsorted(os.listdir(path1)):
    str1 = str(images)
    img1 = os.path.join(path1, images)
    #print(img1)
    img1 = cv2.imread(img1)
    img1 = img_enhance(img1)
    kp1, des1 = akaze.detectAndCompute(img1,None)
    if len(kp1)>0:
       for image in natsorted(os.listdir(path1)):
           img2 = os.path.join(path1, image)
           str2 = str(image)
           if (str1.split("_")[2]==str2.split("_")[2]):
               #print(str1.split("_"))
               continue
           if (img1 != img2):
               img2 = cv2.imread(img2)
               img2 = img_enhance(img2)
               kp2, des2 = akaze.detectAndCompute(img2,None)
          
               if des1 is None:
                 file1.write( str1 + " " + str2 + " " + "0" +"\n")
                 print(str1 + " " + str2 + " " + str(score))
                 continue
               if des2 is None:
                 file1.write( str1 + " " + str2 + " " + "0"+"\n")
                 print(str1 + " " + str2 + " " + str(score))
                 continue
              #  print(des1)
              #  print(des2)
              #  print(type(des1),type(des2))
               matches = bf.knnMatch(des1,des2, k=2)
               dis = bf.match(des1,des2)
            #----------------------------------------------
               number_keypoints = 0
               if len(kp1) <= len(kp2):
                   number_keypoints = len(kp1)
               else:
                   number_keypoints = len(kp2)
               good = []
               #print(matches)
               try:
                 for m,n in matches:
                     if m.distance < 0.9*n.distance:
                         good.append([m])
               except:
                 pass 
               score = len(good) / number_keypoints
               if(score>1):
                 file1.write( str1 + " " + str2 + " " + "1.0"+"\n")
                 print(str1 + " " + str2 + " " + "1.0")
               else:
                 file1.write( str1 + " " + str2 + " " + str(score)+"\n")
                 print(str1 + " " + str2 + " " + str(score))

In [ ]:
file1.close()
genuine = []
imposter = []
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/output_akaze.csv", header=None)
for i in df[0]:
  print(i)
  str1 = i.split()
  a = str1[0].split("_") 
  b = str1[1].split("_")
  if (a[0]==b[0]):
    genuine.append(float(str1[2]))
  else:
    imposter.append(float(str1[2]))

In [ ]:
bins = np.linspace(0, 1, 100)
from matplotlib import pyplot
pyplot.hist(imposter, bins, alpha=0.25, label='imposter', weights=np.ones(len(imposter)) / len(imposter), histtype = 'stepfilled', align = 'mid')
pyplot.hist(genuine, bins, alpha=0.5, label='genuine', weights=np.ones(len(genuine)) / len(genuine), histtype = 'stepfilled')
pyplot.legend(loc='upper right')
pyplot.show()

In [ ]:
scores_akaze = {}
crr_akaze = []
far_akaze = []
frr_akaze = []
eer_thresh_akaze = 1000
eer_diff_akaze = 1000
crrAkaze = 0
for thresh in threshes:
    scores_akaze[thresh] = calc_scores(thresh, genuine, imposter)
    crr_akaze.append(scores_akaze[thresh][0])
    far_akaze.append(scores_akaze[thresh][1])
    frr_akaze.append(scores_akaze[thresh][2])
    if(abs(scores_akaze[thresh][1]-scores_akaze[thresh][2])<eer_diff_akaze):
      eer_diff_akaze = abs(scores_akaze[thresh][1]-scores_akaze[thresh][2])
      eer_thresh_akaze = thresh
      crrAkaze = scores_akaze[thresh][0]
plt.plot(far_akaze, frr_akaze)
plt.xlabel('False Acceptance Rate')
plt.ylabel('False Rejection Rate')
plt.show()
print("eer_diff=",eer_diff_akaze," eer_thresh=",eer_thresh_akaze)

In [ ]:
crrAkaze

# BRISK



In [ ]:
file1 = open("/content/drive/MyDrive/Colab Notebooks/output_brisk.csv", "w")
path1 = '/content/drive/MyDrive/Colab Notebooks/forehead_dataset/'

In [ ]:
brisk = cv2.BRISK_create()
bf = cv2.BFMatcher()
for images in natsorted(os.listdir(path1)):
    str1 = str(images)
    img1 = os.path.join(path1, images)
    #print(img1)
    img1 = cv2.imread(img1)
    img1 = img_enhance(img1)
    kp1, des1 = brisk.detectAndCompute(img1,None)
    if len(kp1)>0:
       for image in natsorted(os.listdir(path1)):
           img2 = os.path.join(path1, image)
           str2 = str(image)
           if (str1.split("_")[2]==str2.split("_")[2]):
               #print(str1.split("_"))
               continue
           if (img1 != img2):
               img2 = cv2.imread(img2)
               img2 = img_enhance(img2)
               kp2, des2 = brisk.detectAndCompute(img2,None)
          
               if des1 is None:
                 file1.write( str1 + " " + str2 + " " + "0" +"\n")
                 print(str1 + " " + str2 + " " + str(score))
                 continue
               if des2 is None:
                 file1.write( str1 + " " + str2 + " " + "0"+"\n")
                 print(str1 + " " + str2 + " " + str(score))
                 continue
              #  print(des1)
              #  print(des2)
              #  print(type(des1),type(des2))
               matches = bf.knnMatch(des1,des2, k=2)
               dis = bf.match(des1,des2)
            #----------------------------------------------
               number_keypoints = 0
               if len(kp1) <= len(kp2):
                   number_keypoints = len(kp1)
               else:
                   number_keypoints = len(kp2)
               good = []
               #print(matches)
               try:
                 for m,n in matches:
                     if m.distance < 0.9*n.distance:
                         good.append([m])
               except:
                 pass 
               score = len(good) / number_keypoints
               if(score>1):
                 file1.write( str1 + " " + str2 + " " + "1.0"+"\n")
                 print(str1 + " " + str2 + " " + "1.0")
               else:
                 file1.write( str1 + " " + str2 + " " + str(score)+"\n")
                 print(str1 + " " + str2 + " " + str(score))

In [ ]:
file1.close()
genuine = []
imposter = []
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/output_brisk.csv", header=None)
for i in df[0]:
  print(i)
  str1 = i.split()
  a = str1[0].split("_") 
  b = str1[1].split("_")
  if (a[0]==b[0]):
    genuine.append(float(str1[2]))
  else:
    imposter.append(float(str1[2]))

In [ ]:
bins = np.linspace(0, 1, 100)
from matplotlib import pyplot
pyplot.hist(imposter, bins, alpha=0.25, label='imposter', weights=np.ones(len(imposter)) / len(imposter), histtype = 'stepfilled', align = 'mid')
pyplot.hist(genuine, bins, alpha=0.5, label='genuine', weights=np.ones(len(genuine)) / len(genuine), histtype = 'stepfilled')
pyplot.legend(loc='upper right')
pyplot.show()

In [ ]:
scores_brisk = {}
crr_brisk = []
far_brisk = []
frr_brisk = []
eer_thresh_brisk = 1000
eer_diff_brisk = 1000
crrBrisk = 0
for thresh in threshes:
    scores_brisk[thresh] = calc_scores(thresh, genuine, imposter)
    crr_brisk.append(scores_brisk[thresh][0])
    far_brisk.append(scores_brisk[thresh][1])
    frr_brisk.append(scores_brisk[thresh][2])
    if(abs(scores_brisk[thresh][1]-scores_brisk[thresh][2])<eer_diff_brisk):
      eer_diff_brisk = abs(scores_brisk[thresh][1]-scores_brisk[thresh][2])
      eer_thresh_brisk = thresh
      crrBrisk = scores_brisk[thresh][0]
plt.plot(far_brisk, frr_brisk)
plt.xlabel('False Acceptance Rate')
plt.ylabel('False Rejection Rate')
plt.show()
print("eer_diff=",eer_diff_brisk," eer_thresh=",eer_thresh_brisk)

In [ ]:
crrBrisk

# SSIM

In [ ]:
from skimage.metrics import structural_similarity as compare_ssim


In [ ]:
file1 = open("/content/drive/MyDrive/Colab Notebooks/output_ssim.csv", "w")
path1 = '/content/drive/MyDrive/Colab Notebooks/forehead_dataset/'

In [ ]:
for images in natsorted(os.listdir(path1)):
    str1 = str(images)
    if str1=='.ipynb_checkpoints':
        continue
    img1 = os.path.join(path1, images)
    img1 = cv2.imread(img1)
    img1 = img_enhance(img1)
    for image in natsorted(os.listdir(path1)):
        img2 = os.path.join(path1, image)
        str2 = str(image)
        if str1 == '.ipynb_checkpoints' or str2 == '.ipynb_checkpoints':
            continue
        if (str1.split("_")[2]==str2.split("_")[2]):
            continue
        if (img1 != img2):
            img2 = cv2.imread(img2)
            img2 = img_enhance(img2)
            img2=cv2.resize(img2,(img1.shape[1],img1.shape[0]))
            (score, diff) = compare_ssim(img1, img2, full=True)
            diff = (diff * 255).astype("uint8")       
            file1.write(str1 + " " + str2 + " " + str(score)+"\n")
            print(str1 + " " + str2 + " " + str(score))

In [ ]:
file1.close()
genuine = []
imposter = []
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/output_ssim.csv", header=None)
for i in df[0]:
  print(i)
  str1 = i.split()
  a = str1[0].split("_") 
  b = str1[1].split("_")
  if (a[0]==b[0]):
    genuine.append(float(str1[2]))
  else:
    imposter.append(float(str1[2]))

In [ ]:
norm_gen = [(float(i)-min(genuine))/(max(genuine)-min(genuine)) for i in genuine]
norm_imp = [(float(i)-min(imposter))/(max(imposter)-min(imposter)) for i in imposter]

In [ ]:
bins = np.linspace(0, 1, 100)
from matplotlib import pyplot
pyplot.hist(imposter, bins, alpha=0.25, label='imposter', weights=np.ones(len(imposter)) / len(imposter), histtype = 'stepfilled', align = 'mid')
pyplot.hist(genuine, bins, alpha=0.5, label='genuine', weights=np.ones(len(genuine)) / len(genuine), histtype = 'stepfilled')
pyplot.legend(loc='upper right')
pyplot.show()

In [ ]:
scores_ssim = {}
crr_ssim = []
far_ssim = []
frr_ssim = []
eer_thresh_ssim = 1000
eer_diff_ssim = 1000
crrSsim = 0
for thresh in threshes:
    scores_ssim[thresh] = calc_scores(thresh, genuine, imposter)
    crr_ssim.append(scores_ssim[thresh][0])
    far_ssim.append(scores_ssim[thresh][1])
    frr_ssim.append(scores_ssim[thresh][2])
    if(abs(scores_ssim[thresh][1]-scores_ssim[thresh][2])<eer_diff_ssim):
      eer_diff_ssim = abs(scores_ssim[thresh][1]-scores_ssim[thresh][2])
      eer_thresh_ssim = thresh
      crrSsim = scores_ssim[thresh][0]
plt.plot(far_ssim, frr_ssim)
plt.xlabel('False Acceptance Rate')
plt.ylabel('False Rejection Rate')
plt.show()
print("eer_diff=",eer_diff_ssim," eer_thresh=",eer_thresh_ssim)

In [ ]:
crrSsim

# Combined ROC curve

In [ ]:
plt.plot(far_sift,frr_sift, label="SIFT")
plt.plot(far_surf,frr_surf, label="SURF")
plt.plot(far_arc,frr_arc, label="ArcFace")
plt.plot(far_orb,frr_orb, label="ORB")
plt.plot(far_akaze,frr_akaze, label="AKAZE")
plt.plot(far_brisk,frr_brisk, label="BRISK")
plt.plot(far_ssim,frr_ssim, label="SSIM")
plt.xlabel('False Acceptance Rate')
plt.ylabel('False Rejection Rate')
plt.legend(loc='upper right')